# Bright SNe lightcurves

This is based on the Lasair example at https://github.com/lsst-uk/lasair-examples/blob/main/notebooks/API_lsst/BrightSNe.ipynb

In [ ]:
import getpass
from lasair import lasair_client as lasair

from panoptes_client import SubjectSet, Panoptes

from src.lsst_zooniverse.generator import lightcurve_to_subject

In [ ]:
zooniverse_username = input('Zooniverse username')
zooniverse_password = getpass.getpass('Zooniverse password')
API_TOKEN = getpass.getpass('Lasair API token')

In [ ]:
Panoptes.connect(username=zooniverse_username, password=zooniverse_password)

In [ ]:
PROJECT_ID = 7
SUBJECT_SET_ID = 132831

In [ ]:
endpoint = "https://api.lasair-lsst.lsst.ac.uk/api"
# endpoint = "https://lasair-ztf.lsst.ac.uk/api"

In [ ]:
L = lasair(API_TOKEN, endpoint=endpoint)

In [ ]:
ra = 51.358273
dec= -27.692442
radii = [100.0, 10, 0]
for radius in radii:
    result = L.cone(ra, dec, radius, requestType='count')
    print('Found %d at radius %f' % (result['count'], radius))

In [ ]:
# Just join the objects table and the sherlock_classifications table, just
# pulling out the objectId.
rows = L.query(
    "objects.diaObjectId AS objectId",
    'objects, sherlock_classifications',
    """ sherlock_classifications.classification = "SN"
        AND objects.g_psfFlux > 2000 AND objects.r_psfFlux > 2000
        AND objects.nSources > 1 
    """,
    limit=10)
objectIds = [row['objectId'] for row in rows]
print(objectIds)

In [ ]:
bandColors = {'u': '#9900cc', 'g': '#3366ff', 'r':'#33cc33', 'i':'#ffcc00', 'z':'#ff0000', 'y':'#cc6600'}
glyphs = tuple(('circle', c) for c in bandColors.values())

In [ ]:
subjects = []
for obj in objectIds:
    lc = L.object(obj, lasair_added=False)
    subjects.append(lightcurve_to_subject(lc['diaSources'], project=PROJECT_ID, metadata={'object ID': obj}, glyphs=glyphs))
SubjectSet(SUBJECT_SET_ID).add(subjects)